# Calibration

Probabilistic numerical finite differences use the formalism of Gaussian process regression to derive the schemes.
This brings with it the advantage of uncertainty quantification, but also the burden of choosing a useful prior model.

In this notebook, we will discuss the basics of model selection and uncertainty quantification.


In [1]:
import functools

import jax
import jax.numpy as jnp
import jax.scipy.stats
from jax.config import config

import probfindiff
from probfindiff import stencil
from probfindiff.utils import autodiff, kernel, kernel_zoo

config.update("jax_enable_x64", True)

First, a baseline. Since the function has a large output-scale, the error and uncertainty quantification are off.

In [2]:
dx = 0.01
order_method = 5
scheme, xs = probfindiff.central(
    dx=dx, kernel=kernel_zoo.exponentiated_quadratic, order_method=order_method
)

f = lambda x: 100 * jnp.sin(x)
fx = f(xs)
dfx, variance = probfindiff.differentiate(fx, scheme=scheme)

dfx_true = jax.grad(f)(0.0)
error, std = jnp.abs(dfx - dfx_true), jnp.sqrt(variance)
print("Error:\n\t", error)
print("Standard deviation:\n\t", std)

Error:
	 4.719146676279706e-05
Standard deviation:
	 9.506929163641619e-06


We can tune the prior kernel to alleviate this issue.
For example, we can compute some maximum-likelihood estimate of input-scale and output-scale.
The goal is to find
$$
\arg\max_{a, b} p(f_{a,b}(x_n) = f_n, ~ n=0, ..., N)
$$
where $f$ is the prior Gaussian process, $f_n$ are the observations of the to-be-differentiated function, and $x_n$ are the finite difference grid points.

In [3]:
def kernel_from_scale(*, input_scale, output_scale):
    k = functools.partial(
        kernel_zoo.exponentiated_quadratic,
        input_scale=input_scale,
        output_scale=output_scale,
    )
    return kernel.batch_gram(k)


def logpdf_from_kernel(*, kernel, x, fx):
    K = kernel(x[:, None], x[None, :])
    return jax.scipy.stats.multivariate_normal.logpdf(
        fx, mean=jnp.zeros_like(fx), cov=K
    )


@jax.jit
def logpdf_from_scale(*, input_scale, output_scale, x, fx):
    k, *_ = kernel_from_scale(input_scale=input_scale, output_scale=output_scale)
    return logpdf_from_kernel(kernel=k, x=x, fx=fx)

The problem is very small, so let us be extremely lazy and compute some grid-search over a logarithmic space to find the solution of the above optimisation problem.

In [14]:
l_max, i_max, o_max = -jnp.inf, None, None

Is = jnp.logspace(-5, 3, num=100, endpoint=True)
Os = jnp.logspace(-4, 2, num=100, endpoint=True)

res = jnp.asarray(
    [
        [logpdf_from_scale(input_scale=i, output_scale=o, x=xs, fx=fx) for i in Is]
        for o in Os
    ]
)

k, l = jnp.unravel_index(jnp.argmax(res, axis=None), res.shape)
k_optimal = functools.partial(
    kernel_zoo.exponentiated_quadratic,
    input_scale=Is[k],
    output_scale=Os[l],
)
print(f"Optimal parameters:\n\tinput_scale={Is[k]}\n\toutput_scale={Os[l]}")

Optimal parameters:
	input_scale=1e-05
	output_scale=0.0001


In [15]:
scheme, xs = probfindiff.central(dx=dx, kernel=k_optimal, order_method=order_method)
dfx, variance = probfindiff.differentiate(f(xs), scheme=scheme)
error, std = jnp.abs(dfx - dfx_true), jnp.sqrt(variance)
print("Error:\n\t", error)
print("Standard deviation:\n\t", std)

Error:
	 0.9957098954750734
Standard deviation:
	 3.1465848318198807e-06
